# Process Raw Data #

### Run DADA2 (v1.8) ###
https://benjjneb.github.io/dada2/tutorial_1_8.html

In [ ]:
library(dada2); packageVersion("dada2")

In [ ]:
path <- "/Users/mcarrion/Korem_Lab/combined/data" 
list.files(path)

# files should be of the form:
# orig_1001A-D0_S25_1.fastq
# orig_1001A-D0_S25_2.fastq
# ...

In [ ]:
# Forward and reverse fastq filenames have format: _1.fastq and _2.fastq, respectively
fnFs <- sort(list.files(path, pattern="_1.fastq", full.names = TRUE))
fnRs <- sort(list.files(path, pattern="_2.fastq", full.names = TRUE))

# Extract sample names
sample.names <- sapply(strsplit(sub("\\.fastq$|\\.fq$", "", basename(fnFs)), "_"), function(x) {
  # If the last element is "1" or "2", remove it
  if (x[length(x)] %in% c("1", "2")) {
    x <- x[-length(x)]  # Drop last element
  }
  return(paste(x, collapse = "_"))  # Reconstruct name
})

In [ ]:
plotQualityProfile(fnFs[21:22])


In [ ]:
# Place filtered files in filtered/ subdirectory
filtFs <- file.path(path, "filtered", paste0(sample.names, "_F_filt.fastq.gz"))
filtRs <- file.path(path, "filtered", paste0(sample.names, "_R_filt.fastq.gz"))

In [ ]:
out <- filterAndTrim(fnFs, filtFs, fnRs, filtRs, truncLen=c(280,240),
              maxN=0, maxEE=c(2,2), truncQ=2, rm.phix=FALSE,
              compress=TRUE, multithread=TRUE) # On Windows set multithread=FALSE
head(out)

In [ ]:
errF <- learnErrors(filtFs, multithread=TRUE)


In [ ]:
errR <- learnErrors(filtRs, multithread=TRUE)


In [ ]:
plotErrors(errF, nominalQ=TRUE)


In [ ]:
derepFs <- derepFastq(filtFs, verbose=TRUE)
names(derepFs) <- sample.names

In [ ]:
derepRs <- derepFastq(filtRs, verbose=TRUE)
names(derepRs) <- sample.names

In [ ]:
dadaFs <- dada(derepFs, err=errF, multithread=TRUE)


In [ ]:
dadaRs <- dada(derepRs, err=errR, multithread=TRUE)


In [ ]:
mergers <- mergePairs(dadaFs, derepFs, dadaRs, derepRs, verbose=TRUE)

# Inspect the merger data.frame from the first sample
head(mergers[[1]])

In [ ]:
seqtab <- makeSequenceTable(mergers)

In [ ]:
seqtab.nochim <- removeBimeraDenovo(seqtab, method="consensus", multithread=TRUE, verbose=TRUE)
dim(seqtab.nochim)

In [ ]:
getN <- function(x) sum(getUniques(x))
track <- cbind(out, sapply(dadaFs, getN), sapply(dadaRs, getN), sapply(mergers, getN), rowSums(seqtab.nochim))
# If processing a single sample, remove the sapply calls: e.g. replace sapply(dadaFs, getN) with getN(dadaFs)
colnames(track) <- c("input", "filtered", "denoisedF", "denoisedR", "merged", "nonchim")
rownames(track) <- sample.names
head(track)

In [ ]:
# filtered_seqtab obtained from below
# in reality, we just get colnames/rownames, so can do it using seqtab.nochim above too
filtered_seqtab2 <- filtered_seqtab %>%
	select(-any_of(c("Sample_ID", "filtered", "denoisedF", "denoisedR", "merged", "nonchim")))
rownames(filtered_seqtab2) <- filtered_seqtab2[, "Sample"]  # Assign sample names as row names
filtered_seqtab2 <- filtered_seqtab2[, -1, drop=FALSE]  # Remove "Sample" column
filtered_seqtab2 <- t(filtered_seqtab2)
head(filtered_seqtab2)


In [ ]:
# NB: taxonomy data can be downloaded from dada2 link above
taxa <- assignTaxonomy(rownames(filtered_seqtab2), "/Users/mcarrion/Korem_Lab/validation_data/silva_nr_v128_train_set.fa.gz", multithread=TRUE)


In [ ]:
taxa <- addSpecies(taxa, "/Users/mcarrion/Korem_Lab/validation_data/silva_species_assignment_v128.fa.gz")


In [ ]:
path <- file.path("/Users/mcarrion/Korem_Lab/combined")

write.csv(taxa, file = file.path(path, "taxa.csv"), row.names=TRUE)

In [ ]:
taxa.print <- taxa # Removing sequence rownames for display only
rownames(taxa.print) <- NULL
head(taxa.print)

In [ ]:
path <- file.path("/Users/mcarrion/Korem_Lab/combined")

# Save CSV files in the specified folder
write.csv(seqtab.nochim, file = file.path(path, "seqtab_nochim.csv"), row.names=TRUE)
write.csv(track, file = file.path(path, "track.csv"), row.names=TRUE)

### Removing Duplicates ###

In [ ]:
library(tidyverse)

# ** Read and fix the track file**
read_and_fix_track <- function(filepath) {
  df <- read.csv(filepath, check.names = FALSE)  # Read CSV without altering names

  # Ensure the first column is named "Sample"
  if (colnames(df)[1] == "" || colnames(df)[1] == "X") {
    colnames(df)[1] <- "Sample"
  }

  return(df)
}

# Read track.csv from orig_data directory
#track_combined <- read_and_fix_track("/Users/mcarrion/Korem_Lab/combined/track.csv")
#track_combined <- track

# ** Extract Sample ID (everything before first `_`)**
track_combined <- track_combined %>%
  mutate(
    Sample_ID = str_extract(Sample, "^[^_]+_[^_]+")  # Extract part before first `_`
  )

# ** Keep only the row with the highest 'nonchim' value per Sample_ID**
best_samples <- track_combined %>%
  group_by(Sample_ID) %>%
  slice_max(nonchim, with_ties = FALSE) %>%
  ungroup()

print("Best samples selected successfully!")
print(best_samples)

# ** Read and fix the sequence table**
#seqtab <- read.csv("/Users/mcarrion/Korem_Lab/combined/seqtab_nochim.csv",
#                   row.names = 1, check.names = FALSE) %>%
#  rownames_to_column("Sample")

# ** Extract Sample ID from Sequence Table**
seqtab <- seqtab %>%
  mutate(
    Sample_ID = str_extract(Sample, "^[^_]+_[^_]+")  # Extract part before first `_`
  )

# ** Filter sequence table to keep only the best Sample_IDs**
filtered_seqtab <- seqtab %>%
  filter(Sample %in% best_samples$Sample)

# **Ensure "Sample" is the identifier column**
colnames(filtered_seqtab)[1] <- "Sample"

# **Save the deduplicated sequence table**
#write.csv(filtered_seqtab, "/Users/mcarrion/Korem_Lab/orig_data/deduplicated_seqtab_nochim.csv", row.names = FALSE)
#print("Filtered sequence table saved successfully!")

### Removing Erroneous Samples ###

In [ ]:
filtered_seqtab <- filtered_seqtab %>% filter(Sample != "val_SB157_S244")
# print("Checking if SB157 still exists in filtered_seqtab:")
print("val_SB157_S244" %in% filtered_seqtab$Sample)  # Should return FALSE
# write.csv(filtered_seqtab, "/Users/mcarrion/Korem_Lab/data/deduplicated_seqtab_nochim.csv", row.names = FALSE)

In [ ]:
# Remove extra columns used for de-duplication 
#filtered_seqtab <- filtered_seqtab %>%
#  select(-any_of(c("input", "filtered", "denoisedF", "denoisedR", "merged", "nonchim")))
#write.csv(filtered_seqtab, "/Users/mcarrion/Korem_Lab/data/deduplicated_seqtab_nochim.csv", row.names = FALSE)

### See how much overlap exists between two cohorts ###

In [ ]:
filtered_seqtab <- read.csv("/Users/mcarrion/Korem_Lab/combined/deduplicated_seqtab_nochim.csv",
                  row.names = 1, check.names = FALSE)

In [ ]:
filtered_seqtab <- filtered_seqtab[, !colnames(filtered_seqtab) %in% c("Sample_ID"), drop = FALSE]

head(filtered_seqtab)

In [ ]:
# Extract row names (sample names)
sample_names <- rownames(filtered_seqtab)

# Identify orig_ and val_ sample rows
orig_rows <- grep("^orig_", sample_names, value = TRUE)
val_rows  <- grep("^val_", sample_names, value = TRUE)

# Subset ASV abundances for orig_ and val_ samples (keeping only numeric ASV columns)
orig_abundance <- filtered_seqtab[orig_rows, , drop = FALSE]
val_abundance  <- filtered_seqtab[val_rows, , drop = FALSE]

# Ensure all ASV columns are numeric (in case of any issues)
orig_abundance[] <- lapply(orig_abundance, as.numeric)
val_abundance[]  <- lapply(val_abundance, as.numeric)

# Identify ASVs present in at least one sample in each group
asvs_in_orig <- colSums(orig_abundance > 0) > 0
asvs_in_val  <- colSums(val_abundance > 0) > 0

# Find ASVs unique to each group
unique_asvs_orig <- asvs_in_orig & !asvs_in_val  # ASVs in orig_ but NOT in val_
unique_asvs_val <- asvs_in_val & !asvs_in_orig  # ASVs in val_ but NOT in orig_

# Compute total read counts for each unique ASV
reads_unique_orig <- colSums(orig_abundance[, unique_asvs_orig, drop = FALSE])
reads_unique_val  <- colSums(val_abundance[, unique_asvs_val, drop = FALSE])

# Compute total reads for entire orig_ and val_ groups
total_reads_orig <- sum(orig_abundance)
total_reads_val  <- sum(val_abundance)

# Compute proportion of total reads from unique ASVs
prop_reads_unique_orig <- sum(reads_unique_orig) / total_reads_orig
prop_reads_unique_val  <- sum(reads_unique_val) / total_reads_val

# Compute proportion of each unique ASV relative to total reads in orig_/val_
prop_each_unique_orig <- reads_unique_orig / total_reads_orig
prop_each_unique_val  <- reads_unique_val / total_reads_val

# Sort ASVs by highest proportion
sorted_orig <- sort(prop_each_unique_orig, decreasing = TRUE)
sorted_val <- sort(prop_each_unique_val, decreasing = TRUE)

# Convert to data frames for easy viewing
top_unique_orig <- data.frame(ASV = names(sorted_orig), Proportion = sorted_orig)
top_unique_val <- data.frame(ASV = names(sorted_val), Proportion = sorted_val)

# Print total proportion of unique ASVs
cat("Proportion of total reads from unique ASVs in orig_:", prop_reads_unique_orig, "\n")
cat("Proportion of total reads from unique ASVs in val_:", prop_reads_unique_val, "\n")

# Print top 10 ASVs contributing the most reads among unique ASVs
cat("\nTop 10 ASVs unique to orig_ samples (as proportion of total orig_ reads):\n")
print(head(top_unique_orig, 10))

cat("\nTop 10 ASVs unique to val_ samples (as proportion of total val_ reads):\n")
print(head(top_unique_val, 10))

overlapping_asvs <- asvs_in_orig & asvs_in_val  # Logical vector for intersecting ASVs

# Create a new table with only overlapping ASVs
filtered_seqtab_overlap <- filtered_seqtab[, overlapping_asvs, drop = FALSE]

# Compute total reads in full dataset and in overlapping ASVs
total_reads_all <- sum(filtered_seqtab)  # Sum all ASV reads
total_reads_overlap <- sum(filtered_seqtab_overlap)  # Sum reads from intersecting ASVs

# Compute proportion of reads from intersecting ASVs
prop_reads_overlap <- total_reads_overlap / total_reads_all

# Print results
cat("Dimensions of filtered_seqtab:", dim(filtered_seqtab), "\n")
cat("Dimensions of filtered_seqtab_overlap:", dim(filtered_seqtab_overlap), "\n")
cat("Proportion of overlapping ASVs:", dim(filtered_seqtab_overlap)[2] / dim(filtered_seqtab)[2], "\n")
cat("Proportion of total reads from overlapping ASVs:", prop_reads_overlap, "\n")

In [ ]:

cat("ASVs in Orig:", sum(asvs_in_orig), "\n")
cat("ASVs in Val:",sum(asvs_in_val), "\n")
cat("ASVs in Both:",sum(overlapping_asvs), "\n")


In [ ]:
# Calculate how many samples each ASV is present in
asv_presence_counts <- colSums(filtered_seqtab_overlap > 0)

# Determine threshold: 10% of total samples
sample_threshold <- 0.10 * nrow(filtered_seqtab_overlap)

# Count how many ASVs are present in at least 10% of samples
num_asvs_10pct <- sum(asv_presence_counts >= sample_threshold)

# Print result
num_asvs_10pct

In [ ]:
#head(filtered_seqtab_overlap)
#write.csv(filtered_seqtab_overlap, "/Users/mcarrion/Korem_Lab/combined/filtered_seqtab_overlap.csv", row.names = TRUE)

In [ ]:
library(ggplot2)

# Compute total reads per ASV across all samples
total_reads_per_asv <- colSums(filtered_seqtab)

# Compute proportion of total reads per ASV
prop_reads_per_asv <- total_reads_per_asv / sum(total_reads_per_asv)

# Create a data frame for plotting
asv_data <- data.frame(
  ASV = names(prop_reads_per_asv),
  Proportion = prop_reads_per_asv,
  Overlap_Status = ifelse(names(prop_reads_per_asv) %in% names(overlapping_asvs[overlapping_asvs]), 
                          "Overlapping", "Unique")
)

# Sort by proportion for better visualization
asv_data <- asv_data[order(-asv_data$Proportion), ]

# Generate scatter plot
ggplot(asv_data, aes(x = seq_along(ASV), y = Proportion, color = Overlap_Status)) +
  geom_point(alpha = 0.7) +  # Scatter points with transparency
  scale_y_log10() +  # Log scale to highlight differences in abundance
  labs(title = "Proportion of Total Reads per ASV",
       x = "ASV Rank (Sorted by Abundance)",
       y = "Proportion of Total Reads (Log Scale)",
       color = "ASV Type") +
  theme_minimal()

In [ ]:
# Generate box plot
ggplot(asv_data, aes(x = Overlap_Status, y = Proportion, fill = Overlap_Status)) +
  geom_boxplot(outlier.shape = NA, alpha = 0.6) +  # Box plot with no individual outliers
  geom_jitter(width = 0.2, alpha = 0.2) +  # Add slight scatter for visibility
  scale_y_log10() +  # Log scale for better visualization
  labs(title = "Distribution of ASV Read Proportions",
       x = "ASV Type",
       y = "Proportion of Total Reads (Log Scale)",
       fill = "ASV Type") +
  theme_minimal()

In [ ]:
# Generate density plot
ggplot(asv_data, aes(x = Proportion, fill = Overlap_Status)) +
  geom_density(alpha = 0.5, adjust = 1) +  # Density plot with transparency
  scale_x_log10() +  # Log scale for better visualization
  labs(title = "Distribution of ASV Read Proportions",
       x = "Proportion of Total Reads (Log Scale)",
       y = "Density",
       fill = "ASV Type") +
  theme_minimal()

In [ ]:
# library(stringdist)

# # Convert logical vectors to ASV lists
# asvs_orig <- names(asvs_in_orig[asvs_in_orig])  # Extract ASVs present in orig
# asvs_val <- names(asvs_in_val[asvs_in_val])  # Extract ASVs present in val

# # Compute pairwise Levenshtein distances between ASVs
# dist_matrix <- stringdistmatrix(asvs_orig, asvs_val, method = "lv")  # "lv" = Levenshtein

# # Set mismatch threshold (e.g., X = 2)
# X <- 2

# # Identify fuzzy matches: Find ASV pairs where the distance is ≤ X
# fuzzy_matches <- which(dist_matrix <= X, arr.ind = TRUE)

# # Extract matched ASVs
# matched_orig <- asvs_orig[fuzzy_matches[, 1]]
# matched_val <- asvs_val[fuzzy_matches[, 2]]

# # Create a data frame of fuzzy matches
# fuzzy_asv_mapping <- data.frame(orig_ASV = matched_orig, val_ASV = matched_val, distance = dist_matrix[fuzzy_matches])

# # Print results
# cat("Number of fuzzy-matched ASVs:", nrow(fuzzy_asv_mapping), "\n")
# head(fuzzy_asv_mapping)  # Show first few matches

## Run SCRuB for decontamination ##
https://github.com/Shenhav-and-Korem-labs/SCRuB/tree/main/tutorial

In [ ]:
# Check if negative samples have reads (contamination)
filtered_seqtab <- read.csv("/Users/mcarrion/Korem_Lab/combined/filtered_seqtab_overlap.csv",row.names = 1, check.names = FALSE)


# Subset the rows that match target samples
matching_rows <- filtered_seqtab[grep("Neg", rownames(filtered_seqtab),ignore.case = TRUE), , drop = FALSE]

# Compute total read counts (sum across all columns)
total_reads_per_sample <- rowSums(matching_rows)

# Print results
print(total_reads_per_sample)

In [ ]:
devtools::install_github("shenhav-and-korem-labs/SCRuB")


In [ ]:
library(stringr)
library(dplyr)

# Load file list
file_list <- readLines("/Users/mcarrion/Korem_Lab/file_list.txt")

# Define directories (batches)
val_dirs <- c("20241029_16S_FB_ICU", "20241127_16S_FB_ICU", "20241223_16S_FB_ICU")
orig_dirs <- c("20231228_16S_Plate_1", "20231228_16S_Plate_2", "20231228_16S_Plate_3", "20240202_16S_FB")

# Initialize empty batch mapping
sample_to_batch <- list()


clean_sample_name <- function(filename) {
  # Remove L001, _R1_001.fastq.gz, _R2_001.fastq.gz, _1.fastq.gz, _2.fastq.gz
  filename <- str_remove(filename, "_L001")  # Remove _L001 if present
  filename <- str_remove(filename, "_R[12]_001\\.fastq\\.gz$")  # Remove _R1_001.fastq.gz, _R2_001.fastq.gz
  filename <- str_remove(filename, "_[12]\\.fastq\\.gz$")  # Remove _1.fastq.gz, _2.fastq.gz
  return(filename)
}

# Process file paths
for (filepath in file_list) {
    # Extract directory and filename
    path_parts <- unlist(strsplit(filepath, "/"))
    dir_name <- path_parts[length(path_parts) - 1]  # Second to last part (batch directory)
    filename <- path_parts[length(path_parts)]     # Last part (filename)

    # Remove _1/2 and _R1/2_001.fastq.gz from filename
    clean_name <- clean_sample_name(filename)
    # Determine batch type
    if (dir_name %in% val_dirs) {
        sample_to_batch[[paste0("val_", clean_name)]] <- dir_name
    } else if (dir_name %in% orig_dirs) {
        sample_to_batch[[paste0("orig_", clean_name)]] <- dir_name
    }
}

# Convert list to named vector
sample_to_batch <- unlist(sample_to_batch)

# Assign batch labels in filtered_seqtab
filtered_seqtab$Batch <- sample_to_batch[rownames(filtered_seqtab)]

# Remove rows where batch couldn't be determined
print(filtered_seqtab[is.na(filtered_seqtab$Batch), , drop = FALSE])
filtered_seqtab <- filtered_seqtab[!is.na(filtered_seqtab$Batch), ]

# Split filtered_seqtab into batch-specific DataFrames
batch_dfs <- split(filtered_seqtab, filtered_seqtab$Batch)


In [ ]:
library(SCRuB)

# Initialize a list to store results
all_decontaminated_counts <- list()

# Function to apply SCRuB to a batch
apply_scrub_to_batch <- function(test_batch, batch_name) {
  cat("\n Processing batch:", batch_name, "\n")  # Print batch name before processing

  # Convert to matrix, ensuring numeric values
  count_matrix <- as.matrix(test_batch)
  count_matrix <- count_matrix[, colnames(test_batch) != "Batch", drop = FALSE]

  # Convert all columns to numeric safely
  count_matrix <- as.matrix(apply(count_matrix, 2, as.numeric))
  rownames(count_matrix) <- rownames(test_batch)

  # Replace NA values (SCRuB does not accept missing values)
  count_matrix[is.na(count_matrix)] <- 0  

  # Ensure all values are numeric before SCRuB
  if (!all(sapply(count_matrix, is.numeric))) {
    stop(paste("Error: Non-numeric values found in batch:", batch_name))
  }

  # Create metadata
metadata <- data.frame(
  is_control = grepl("ext[-_]neg|pcr[-_]neg|(^|[-_])neg", rownames(count_matrix), ignore.case = TRUE),
  sample_type = case_when(
    grepl("ext[-_]neg", rownames(count_matrix), ignore.case = TRUE) ~ "extraction control",
    grepl("pcr[-_]neg", rownames(count_matrix), ignore.case = TRUE) ~ "pcr control",
    grepl("(^|[-_])neg", rownames(count_matrix), ignore.case = TRUE) ~ "extraction control",
    TRUE ~ "sample"
  )
)
  rownames(metadata) <- rownames(count_matrix)

  # Debugging: Print sample type distribution
  cat("Sample type distribution in batch", batch_name, ":\n")
  print(table(metadata$sample_type))

  # **Filter `control_order` to only include present control types**
  present_controls <- unique(metadata$sample_type[metadata$is_control])
  control_order <- intersect(c("extraction control", "pcr control"), present_controls)

  # **Skip SCRuB if no valid controls exist (e.g., batch `20240202_16S_FB`)**
  if (length(control_order) == 0) {
    cat("Skipping batch:", batch_name, "because no valid controls were found.\n")
    # Store unprocessed counts in final dataset
    all_decontaminated_counts[[batch_name]] <<- count_matrix
    
    return(list(decontaminated_samples = count_matrix, p = rep(NA, nrow(count_matrix))))
  }

  # Debugging: Print final control order used
  cat(" Using control_order for batch", batch_name, ":", control_order, "\n")

  # Apply SCRuB
  scrub_result <- SCRuB(count_matrix, metadata, control_order = control_order)

  # Extract decontaminated counts
  decontaminated_counts <- scrub_result$decontaminated_samples

  # Store results in global list
  all_decontaminated_counts[[batch_name]] <<- decontaminated_counts

  # Ensure output directory exists
  output_dir <- "/Users/mcarrion/Korem_Lab/scrub_results/"
  if (!dir.exists(output_dir)) {
    dir.create(output_dir, recursive = TRUE)
  }

  # Save batch-specific decontaminated matrix
  write.csv(decontaminated_counts, file = paste0(output_dir, batch_name, "_scrubbed.csv"), row.names = TRUE)

  # Save estimated contamination fractions
  write.csv(scrub_result$p, file = paste0(output_dir, batch_name, "_scrub_p.csv"), row.names = TRUE)

  return(scrub_result)  # Return SCRuB result for further analysis if needed
}

# Apply SCRuB to each batch in batch_dfs, skipping batches without controls
scrub_results <- lapply(names(batch_dfs), function(batch) {
  result <- apply_scrub_to_batch(batch_dfs[[batch]], batch)
  if (!is.null(result)) result  # Only keep non-null results
})

# Remove NULL values from `scrub_results`
scrub_results <- scrub_results[!sapply(scrub_results, is.null)]


# Convert results to a named list
names(scrub_results) <- names(batch_dfs)[names(batch_dfs) %in% names(scrub_results)]

# **Merge all decontaminated counts into one DataFrame**
if (length(all_decontaminated_counts) > 0) {
  unified_decontaminated_counts <- do.call(rbind, all_decontaminated_counts)

  # Save the unified decontaminated DataFrame
  write.csv(unified_decontaminated_counts, "/Users/mcarrion/Korem_Lab/scrub_results/unified_scrubbed_counts.csv", row.names = TRUE)

  cat("SCRuB processing complete for all batches!\n")
} else {
  cat("No valid batches processed through SCRuB.\n")
}

In [ ]:
library(ggplot2)

# Function 1: Plot Estimated Proportion of Non-Contaminated Reads (p values)
plot_p_values <- function(scrub_results) {
  # Extract valid p values, skipping batches with all NAs
  p_values_list <- lapply(scrub_results, function(x) {
    if (!is.null(x$p) && length(x$p) > 0 && !all(is.na(x$p))) {
      data.frame(Sample = names(x$p), p = unlist(x$p), stringsAsFactors = FALSE)
    } else {
      return(NULL)  # Skip batches with all NAs
    }
  })

  # Combine non-null results
  p_values <- do.call(rbind, p_values_list)

  # Ensure we have valid data to plot
  if (is.null(p_values) || nrow(p_values) == 0) {
    stop("⚠️ No valid p values available for plotting.")
  }

  # Generate boxplot
  ggplot(p_values, aes(x = "", y = p)) +
    geom_boxplot(fill = "blue", alpha = 0.5) +
    geom_jitter(width = 0.1, alpha = 0.6, color = "black") +
    labs(
      title = "Proportion of Sample NOT Identified as Contamination (p values)", 
      y = "Proportion of Sample Retained (p)", x = "Samples"
    ) +
    theme_minimal()
}

# Function 2: Plot Total Reads Before vs. After SCRuB
plot_total_reads <- function(raw_counts, decontaminated_counts) {
  # Ensure row names (samples) match in both datasets
  common_samples <- intersect(rownames(raw_counts), rownames(decontaminated_counts))

  # Subset both datasets to include only common samples
  raw_counts <- raw_counts[common_samples, , drop = FALSE]
  decontaminated_counts <- decontaminated_counts[common_samples, , drop = FALSE]

  # Compute total reads per sample before & after SCRuB
  total_reads_before <- rowSums(raw_counts, na.rm = TRUE)
  total_reads_after <- rowSums(decontaminated_counts, na.rm = TRUE)

  # Create dataframe for ggplot
  df <- data.frame(
    Sample = rep(common_samples, 2),
    Total_Reads = c(total_reads_before, total_reads_after),
    Status = rep(c("Before SCRuB", "After SCRuB"), each = length(common_samples))
  )

  # Ensure we have valid data to plot
  if (nrow(df) == 0) {
    stop("No valid read count data available for plotting.")
  }

  # Generate bar plot
  ggplot(df, aes(x = Sample, y = Total_Reads, fill = Status)) +
    geom_bar(stat = "identity", position = "dodge") +
    labs(title = "Total Reads Before and After SCRuB", y = "Total Reads", x = "Samples") +
    theme_minimal() +
    theme(axis.text.x = element_blank())  # Hide x-axis labels if too many samples
}

# Merge & Clean Data for Plotting

# Merge all raw batch data
raw_counts <- do.call(rbind, batch_dfs)

# Extract only valid decontaminated samples
valid_decontaminated <- lapply(scrub_results, function(x) {
  if (!is.null(x$decontaminated_samples) && nrow(x$decontaminated_samples) > 0) {
    return(x$decontaminated_samples)
  } else {
    return(NULL)
  }
})

# Merge valid decontaminated datasets
decontaminated_counts <- do.call(rbind, valid_decontaminated)

# Remove "Batch" column if it exists
raw_counts <- raw_counts[, !colnames(raw_counts) %in% "Batch", drop = FALSE]
rownames(raw_counts) <- sub(".*?\\.(orig_|val_)", "\\1", rownames(raw_counts))

decontaminated_counts <- decontaminated_counts[, !colnames(decontaminated_counts) %in% "Batch", drop = FALSE]

# Run Plots
plot_p_values(scrub_results)  # Contamination fractions
plot_total_reads(raw_counts, decontaminated_counts)  # Reads before vs after

In [ ]:
plot_proportion_reads_removed <- function(raw_counts, decontaminated_counts) {
  # Ensure row names (samples) match in both datasets
  common_samples <- intersect(rownames(raw_counts), rownames(decontaminated_counts))

  # Skip plotting if no valid samples exist
  if (length(common_samples) == 0) {
    stop("⚠️ No overlapping samples found between raw and decontaminated data.")
  }

  # Subset both datasets to only the common samples
  raw_counts <- raw_counts[common_samples, , drop = FALSE]
  decontaminated_counts <- decontaminated_counts[common_samples, , drop = FALSE]

  # Compute total reads before & after SCRuB
  total_reads_before <- rowSums(raw_counts, na.rm = TRUE)
  total_reads_after <- rowSums(decontaminated_counts, na.rm = TRUE)

  # Compute proportion of reads removed per sample
  proportion_removed <- (total_reads_before - total_reads_after) / total_reads_before

  # Create dataframe for ggplot
  df <- data.frame(
    Sample = common_samples,
    Proportion_Removed = proportion_removed
  )

  # Ensure we have valid data to plot
  if (nrow(df) == 0) {
    stop("⚠️ No valid read reduction data available for plotting.")
  }

  # Box plot of proportion removed
  ggplot(df, aes(x = "", y = Proportion_Removed)) +
    geom_boxplot(fill = "red", alpha = 0.5) +
    geom_jitter(width = 0.1, alpha = 0.5, color = "black") +
    labs(
      title = "Proportion of Reads Removed Across Samples",
      x = "Samples",
      y = "Proportion of Reads Removed"
    ) +
    theme_minimal()
}
plot_proportion_reads_removed(raw_counts, decontaminated_counts)

## Alpha & Beta Diversity ##

In [ ]:
library(phyloseq)
library(ggplot2)
library(vegan)
library(dplyr)

In [ ]:
combined_seqtab_orig <- read.csv("/Users/mcarrion/Korem_Lab/combined/merged_df_orig.csv", row.names = 1, check.names = FALSE)
combined_taxa <- read.csv("/Users/mcarrion/Korem_Lab/combined/taxa.csv", row.names = 1, check.names = FALSE)
features <- colnames(read.csv("/Users/mcarrion/Korem_Lab/combined/filtered_seqtab_overlap.csv", row.names = 1))

Examine diversity at day 0, preserving independence among samples

In [ ]:
day0_rows <- grep("-D0", rownames(combined_seqtab_orig), value = TRUE)
day0_seqtab <- combined_seqtab_orig[day0_rows, , drop = FALSE]
combined_seqtab_orig <- day0_seqtab  # only consider samples at day 0

In [ ]:
ref_long <- combined_seqtab_orig[, !(colnames(combined_seqtab_orig) %in% features), drop = FALSE]
head(ref_long)

In [ ]:
# Add in batch/plate info
library(stringr)
library(dplyr)

# Load file list
file_list <- readLines("/Users/mcarrion/Korem_Lab/file_list.txt")

# Define directories (batches)
val_dirs <- c("20241029_16S_FB_ICU", "20241127_16S_FB_ICU", "20241223_16S_FB_ICU")
orig_dirs <- c("20231228_16S_Plate_1", "20231228_16S_Plate_2", "20231228_16S_Plate_3", "20240202_16S_FB")

# Initialize empty batch mapping
sample_to_batch <- list()

# Cleaning function for filenames
clean_sample_name <- function(filename) {
  filename <- str_remove(filename, "_L001")
  filename <- str_remove(filename, "_R[12]_001\\.fastq\\.gz$")
  filename <- str_remove(filename, "_[12]\\.fastq\\.gz$")
  filename <- str_match(filename, "([0-9]+[A-Z])")[,2]
  return(filename)
}

# Process file paths
for (filepath in file_list) {
  path_parts <- unlist(strsplit(filepath, "/"))
  dir_name <- path_parts[length(path_parts) - 1]
  filename <- path_parts[length(path_parts)]
  clean_name <- clean_sample_name(filename)
  
  if (dir_name %in% val_dirs) {
    sample_to_batch[[clean_name]] <- paste0("val_", dir_name)
  } else if (dir_name %in% orig_dirs) {
    sample_to_batch[[clean_name]] <- paste0("orig_", dir_name)
  }
}

# Convert list to named vector
sample_to_batch <- unlist(sample_to_batch)

# Add batch column to ref_long
ref_long$batch <- sample_to_batch[ref_long$id]

# View unmatched rows (if any)
unmatched <- ref_long %>% filter(is.na(batch))
print(unmatched)

In [ ]:
head(ref_long)

In [ ]:
# Add in treatment info
ref_long$sample <- rownames(ref_long)
intervention_df <- read.csv("/Users/mcarrion/Korem_Lab/orig_data/intervention_stats.csv", check.names = FALSE)
colnames(intervention_df)[1] <- "id"

ref_long <- left_join(ref_long, intervention_df, by = "id")
colnames(ref_long)[colnames(ref_long) == "arm"] <- "treatment"
rownames(ref_long) <- ref_long$sample
head(ref_long)


In [ ]:
# Additional Data Cleaning
combined_seqtab_orig_filt <- combined_seqtab_orig[, features, drop = FALSE]
otu_table_combined <- otu_table(as.matrix(combined_seqtab_orig_filt), taxa_are_rows = FALSE)
rownames(otu_table_combined) <- gsub("\\.+\\d+$", "", rownames(otu_table_combined)) # Ensure row names align
tax_table_combined <- tax_table(as.matrix(combined_taxa))


# Convert to sample_data object
sample_data_combined <- sample_data(ref_long)

# Only consider common rows between the two tables
common_samples <- intersect(rownames(otu_table_combined), rownames(sample_data_combined))
otu_table_combined <- otu_table_combined[common_samples, , drop=FALSE]
otu_table_combined[is.na(otu_table_combined)] <- 0
sample_data_combined <- sample_data_combined[common_samples, , drop=FALSE]

In [ ]:
physeq_combined <- phyloseq(otu_table_combined, tax_table_combined, sample_data_combined)
colnames(sample_data(physeq_combined)) # to be used in the below

In [ ]:
plot_richness(physeq_combined, x="death", measures=c("Shannon", "Simpson"))

In [ ]:
# Build phyloseq object
physeq_combined <- phyloseq(
  otu_table(otu_table_combined, taxa_are_rows = FALSE),
  tax_table_combined,
  sample_data_combined
)

# Estimate alpha diversity
alpha_df <- estimate_richness(physeq_combined, measures = "Shannon")
alpha_df$sample <- rownames(alpha_df)

# Merge with sample metadata
alpha_df <- left_join(alpha_df, as.data.frame(sample_data_combined), by = "sample")

# Filter for day 0 only
alpha_df_day0 <- alpha_df

# Plot boxplot
ggplot(alpha_df_day0, aes(x = factor(infection), y = Shannon, fill = factor(infection))) +
  geom_boxplot(outlier.shape = NA, alpha = 0.6) +
  geom_jitter(width = 0.2, alpha = 0.5, color = "black") +
  scale_x_discrete(labels = c("No Infection", "Infection")) +
  labs(
    x = "Infection Status",
    y = "Shannon Diversity",
    title = "Shannon Diversity at ICU Admission"
  ) +
  theme_minimal(base_size = 20) +
  theme(
	plot.title = element_text(hjust = 0.5),
	axis.text.x = element_text(size = 23)  # Increase x-axis label size
	) +
  guides(fill = FALSE)

In [ ]:
wilcox.test(Shannon ~ infection, data = alpha_df_day0)$p.value

In [ ]:
# Compute p-value
p_val <- wilcox.test(Shannon ~ death, data = alpha_df_day0)$p.value
p_label <- paste0("p = ", signif(p_val, 2))

# Set y-position for annotation
y_max <- max(alpha_df_day0$Shannon, na.rm = TRUE)
y_line <- y_max + 0.1
y_text <- y_max + 0.2

# Plot
ggplot(alpha_df_day0, aes(x = factor(death), y = Shannon, fill = factor(death))) +
  geom_boxplot(outlier.shape = NA, alpha = 0.6) +
  geom_jitter(width = 0.2, alpha = 0.5, color = "black") +
  scale_x_discrete(labels = c("Survived", "Died")) +
  labs(
    x = "Mortality Status",
    y = "Shannon Diversity at ICU Admission"
  ) +
  ggtitle("Shannon diversity at ICU admission does not differ by mortality status") +
  theme_minimal(base_size = 18) +
  theme(
    plot.title = element_text(hjust = 0.5, size = 15),
    axis.text.x = element_text(size = 16),
    axis.title = element_text(size = 16)
  ) +
  guides(fill = FALSE) +
  annotate("segment", x = 1, xend = 2, y = y_line, yend = y_line, size = 0.6) +
  annotate("text", x = 1.5, y = y_text, label = p_label, size = 5)

In [ ]:
ggplot(alpha_df_day0, aes(x = factor(death), y = Shannon, fill = factor(death))) +
  geom_boxplot(outlier.shape = NA, alpha = 0.6) +
  geom_jitter(width = 0.2, alpha = 0.5, color = "black") +
  scale_x_discrete(labels = c("Survived", "Died")) +
  labs(
    x = "Mortality Status",
    y = "Shannon Diversity",
    title = "Shannon Diversity at ICU Admission"
  ) +
  theme_minimal(base_size = 20) +
  theme(
    plot.title = element_text(hjust = 0.5),
    axis.text.x = element_text(size = 23)  # Increase x-axis label size

  ) +
  guides(fill = FALSE)

In [ ]:
wilcox.test(Shannon ~ death, data = alpha_df_day0)$p.value

In [ ]:
dist_bc <- phyloseq::distance(physeq_combined, method="bray")
ord_nmds <- ordinate(physeq_combined, method="NMDS", distance="bray")

In [ ]:
plot_ordination(physeq_combined, ord_nmds, color="death") +
  ggtitle("NMDS - Bray-Curtis") +
  theme_minimal()

In [ ]:
# Convert infection and death to labeled factors
sample_data(physeq_combined)$infection <- factor(
  sample_data(physeq_combined)$infection, levels = c(0, 1), labels = c("No Infection", "Infection")
)

sample_data(physeq_combined)$death <- factor(
  sample_data(physeq_combined)$death, levels = c(0, 1), labels = c("Survived", "Died")
)

In [ ]:
# Common theme settings using variables
custom_theme <- theme_minimal(base_size = font_main) +
  theme(
    text = element_text(size = font_main),
    axis.title = element_text(size = font_main),
    axis.text = element_text(size = font_secondary),
    legend.title = element_text(size = font_main),
    legend.text = element_text(size = font_secondary),
    plot.title = element_text(hjust = 0.5, size = font_main + 1, face = "bold")
  )


# Extract axis variance (optional)
eig_vals <- ord_pcoa$values$Relative_eig * 100  # percentage variance

# Format axis labels with variance explained
axis1_label <- sprintf("PCoA Axis 1 (%.1f%%)", eig_vals[1])
axis2_label <- sprintf("PCoA Axis 2 (%.1f%%)", eig_vals[2])

# Panel A – Infection
p2a <- plot_ordination(physeq_combined, ord_pcoa, color = "infection") +
  geom_point(size = 3, alpha = 0.8) +
  labs(
    title = expression(beta*"-Diversity Does Not Distinguish Infection Groups"),
    x = axis1_label,
    y = axis2_label,
    color = "Infection"
  ) +
  custom_theme

# Panel B – Mortality
p2b <- plot_ordination(physeq_combined, ord_pcoa, color = "death") +
  geom_point(size = 3, alpha = 0.8) +
  labs(
    title = expression(beta*"-Diversity Does Not Distinguish Mortality Groups"),
    x = axis1_label,
    y = axis2_label,
    color = "Mortality"
  ) +
  custom_theme
  p2
  #p2b


See if there are inherent batch differences / differences in plates

In [ ]:
#Try PCOA 
ord_pcoa <- ordinate(physeq_combined, method="PCoA", distance="bray")
plot_ordination(physeq_combined, ord_pcoa, color="batch") +
  ggtitle("PCoA - Bray-Curtis") +
  theme_minimal()

In [ ]:
library(vegan)

# Compute Bray-Curtis distance matrix
dist_mat <- phyloseq::distance(physeq_combined, method = "bray")

# Extract metadata
meta <- as(sample_data(physeq_combined), "data.frame")

# Run PERMANOVA
adonis2(dist_mat ~ batch, data = meta)

In [ ]:
# Estimate alpha diversity
alpha_df <- estimate_richness(physeq_combined, measures = c("Shannon", "Simpson"))
alpha_df$death <- sample_data(physeq_combined)$death  # add metadata

# Wilcoxon test (for binary outcome)
wilcox.test(Shannon ~ death, data = alpha_df)

# OR Kruskal-Wallis (if death has >2 levels)
kruskal.test(Shannon ~ death, data = alpha_df)


In [ ]:
library(vegan)

# Compute Bray-Curtis distance matrix
dist_mat <- phyloseq::distance(physeq_combined, method = "bray")

# Extract metadata
meta <- as(sample_data(physeq_combined), "data.frame")

# Run PERMANOVA
adonis2(dist_mat ~ death, data = meta)

Examine diversity over time, stratifying by sampleID 

In [ ]:
combined_seqtab_orig <- read.csv("/Users/mcarrion/Korem_Lab/combined/merged_df_orig.csv", row.names = 1, check.names = FALSE)
combined_taxa <- read.csv("/Users/mcarrion/Korem_Lab/combined/taxa.csv", row.names = 1, check.names = FALSE)
features <- colnames(read.csv("/Users/mcarrion/Korem_Lab/combined/filtered_seqtab_overlap.csv", row.names = 1))

In [ ]:
ref_long <- combined_seqtab_orig[, !(colnames(combined_seqtab_orig) %in% features), drop = FALSE]


In [ ]:
# Additional Data Cleaning
combined_seqtab_orig_filt <- combined_seqtab_orig[, features, drop = FALSE]
otu_table_combined <- otu_table(as.matrix(combined_seqtab_orig_filt), taxa_are_rows = FALSE)
rownames(otu_table_combined) <- gsub("\\.+\\d+$", "", rownames(otu_table_combined)) # Ensure row names align
tax_table_combined <- tax_table(as.matrix(combined_taxa))


# Convert to sample_data object
sample_data_combined <- sample_data(ref_long)

# Only consider common rows between the two tables
common_samples <- intersect(rownames(otu_table_combined), rownames(sample_data_combined))
otu_table_combined <- otu_table_combined[common_samples, , drop=FALSE]
otu_table_combined[is.na(otu_table_combined)] <- 0
sample_data_combined <- sample_data_combined[common_samples, , drop=FALSE]

In [ ]:
physeq_combined <- phyloseq(otu_table_combined, tax_table_combined, sample_data_combined)


In [ ]:
library(lme4)
library(lmerTest)

# Estimate alpha diversity
alpha_df <- estimate_richness(physeq_combined, measures = "Shannon")
meta <- as(sample_data(physeq_combined), "data.frame")
meta$subjectID <- sub("-D.*", "", rownames(meta))             # everything before "-D"
meta$day <- as.numeric(gsub(".*-D(\\d+)", "\\1", rownames(meta)))  # number after "-D"

# Merge richness with metadata
alpha_df$subjectID <- meta$subjectID
alpha_df$day <- meta$day
alpha_df$infection <- meta$infection
alpha_df$death <- meta$death


# Fit linear mixed model: Shannon ~ day * death + (1 | subjectID)
lmm <- lmer(Shannon ~ day * death + (1 | subjectID), data = alpha_df)
summary(lmm)

In [ ]:
set.seed(123)  # for reproducibility

otu_table_matrix <- as(otu_table(physeq_combined), "matrix")

# If taxa are rows, transpose
if (taxa_are_rows(physeq_combined)) {
  otu_table_matrix <- t(otu_table_matrix)
}

# Compute Bray-Curtis distance
bray_dist <- vegdist(otu_table_matrix, method = "bray")

# Distance matrix (e.g., Bray-Curtis)
dist_mat <- phyloseq::distance(physeq_combined, method = "bray")

adonis2_result <- adonis2(
  bray_dist ~ infection,
  data = meta,
  permutations = 999,
  strata = meta$subjectID  # block by subject
)

print(adonis2_result)

In [ ]:
# Ensure infection is treated as a categorical variable
alpha_df$infection <- factor(alpha_df$infection, levels = c(0, 1), labels = c("No Infection", "Infection"))

ggplot(alpha_df, aes(x = day, y = Shannon, color = infection, group = infection)) +
  geom_smooth(method = "lm", formula = y ~ x, se = FALSE, size = 1.2) +
  coord_cartesian(ylim = c(0, 5)) +
  labs(
    title = "Shannon Diversity Over Time by Infection Status",
    x = "Day Post-ICU Admission",
    y = "Shannon Diversity",
    color = "Infection Status"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(hjust = 0.5)
  )

In [ ]:
ggplot(alpha_df, aes(x = infection, y = Shannon, fill = infection)) +
  geom_boxplot(alpha = 0.7, outlier.shape = NA) +
  geom_jitter(width = 0.2, alpha = 0.5, color = "black") +
  coord_cartesian(ylim = c(0, 5)) +
  labs(
    title = "Overall Shannon Diversity by Infection Status",
    x = "Infection Status",
    y = "Shannon Diversity"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(hjust = 0.5),
    legend.position = "none"
  )